In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!ls "drive/My Drive/Train_data"

cherry	strawberry  tomato


In [4]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [5]:
!pip install imutils

  Running setup.py bdist_wheel for imutils ... - \ done
  Stored in directory: /root/.cache/pip/wheels/4c/ff/aa/824fb9efc5b8c740d54cd8bc19c7e85fbb8d115c77e56812c7
Successfully built imutils


In [0]:
import numpy as np
import random
import cv2
from imutils import paths
import os
import tensorflow.keras as K
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.preprocessing.image import img_to_array, ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Convolution2D, SeparableConvolution2D, MaxPooling2D, Flatten, Dense, Dropout)

SEED = 309
np.random.seed(SEED)
random.seed(SEED)

Set parameters for your run here. 

In [0]:
# Total train dataset size is 3755
t_size = [3500, 200]
WIDTH, HEIGHT = 300, 300
image_size = (WIDTH, HEIGHT, 3)
test_data_dir = "./drive/My Drive/Train_data"

This collects the indices of the images data ()their file paths)

In [0]:
ps_up = sorted(list(paths.list_images("./drive/My Drive/Train_data")))
ps_up = [ps.replace("\\","") for ps in ps_up]

# build a range as pointers for image files
temp = range(0, len(ps_up))

Load training data

In [0]:
# randomly sample for our training set
# needs to be big enough to be have capacity for a validation set
train_indices = np.random.choice(temp, size = t_size[0], replace = False)
train = [ps_up[i] for i in train_indices]

lbls, imgs = [],[]
for imagePath in train:
  image = cv2.imread(imagePath)
  # load the image, pre-process it, and store it in the data list
  image = cv2.resize(image, (WIDTH, HEIGHT))
  image = img_to_array(image)
  imgs.append(image)
  label = imagePath.split(os.path.sep)[-2]
  lbls.append(label)

X_train = np.array(imgs, dtype="float") / 255.0
y_train = np.array(lbls)

# Binarize the labels
lb = LabelBinarizer()
y_train = lb.fit_transform(y_train)

Load test data

In [0]:
# exclude those training images from our test set
temp = [i for i in temp if i not in train]
# subset for testing
test_indices = np.random.choice(temp, size = t_size[1], replace = False)
test = [ps_up[i] for i in test_indices]

lbls, imgs = [],[]
for imagePath in test:
  image = cv2.imread(imagePath)
  # load the image, pre-process it, and store it in the data list\
  image = cv2.resize(image, (WIDTH, HEIGHT))
  image = img_to_array(image)
  imgs.append(image)
  label = imagePath.split(os.path.sep)[-2]
  lbls.append(label)

X_test = np.array(imgs, dtype="float") / 255.0
y_test = np.array(lbls)

# Binarize the labels
# Re use the original binariser.
y_test = lb.fit_transform(y_test)

This is a quick sanity check

In [11]:
print('X_train', 'y_train', 'X_test', 'y_test')
print([len(X_train)==t_size[0],len(y_train)==t_size[0]],
      [len(X_test)==t_size[1], len(y_test)==t_size[1]])

X_train y_train X_test y_test
[True, True] [True, True]


model

In [0]:
classifier = Sequential()
# Base layer
classifier.add(
    Convolution2D(
        64, 3, 3,
        input_shape=image_size,
        activation='relu'
    )
)
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Convolution2D(128, 3, 3, activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2), padding='same'))

classifier.add(Convolution2D(256, 3, 3, activation='relu', padding='same'))
classifier.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
classifier.add(Convolution2D(512, 3, 3, activation='relu', padding='same'))
classifier.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
classifier.add(Convolution2D(1024, 3, 3, activation='relu', padding='same'))

classifier.add(Flatten())
classifier.add(Dense(units=1024, activation='relu'))
classifier.add(Dense(units=3, activation='sigmoid'))

classifier.compile(
    loss=K.losses.categorical_crossentropy,
    optimizer=K.optimizers.Adam(),
    metrics=['accuracy'],
    
)

Fit without generator:

Callback is CSVLogger so it can write to drive.

In [30]:
classifier.fit(
    X_train, y_train,
    batch_size=32,
    epochs=30,
    verbose=1,
    validation_split=0.2, 
    callbacks=[K.callbacks.CSVLogger('./drive/My Drive/prj/logs/fifth_final.csv', separator=',', append=True)]
)
score = classifier.evaluate(X_test, y_test, verbose=0)
score

Train on 2800 samples, validate on 700 samples
Epoch 1/30
2800/2800 [==============================] - 13s 5ms/step - loss: 1.0731 - acc: 0.4318 - val_loss: 1.0026 - val_acc: 0.5014
Epoch 2/30
2800/2800 [==============================] - 12s 4ms/step - loss: 0.9388 - acc: 0.5643 - val_loss: 0.9008 - val_acc: 0.6000
Epoch 3/30
2800/2800 [==============================] - 11s 4ms/step - loss: 0.8885 - acc: 0.5861 - val_loss: 0.9010 - val_acc: 0.5871
Epoch 4/30
2800/2800 [==============================] - 11s 4ms/step - loss: 0.8515 - acc: 0.6029 - val_loss: 0.9489 - val_acc: 0.5129
Epoch 5/30
2800/2800 [==============================] - 11s 4ms/step - loss: 0.7698 - acc: 0.6618 - val_loss: 0.7889 - val_acc: 0.6671
Epoch 6/30
2800/2800 [==============================] - 11s 4ms/step - loss: 0.7170 - acc: 0.6975 - val_loss: 0.8247 - val_acc: 0.6500
Epoch 7/30
2800/2800 [==============================] - 11s 4ms/step - loss: 0.6608 - acc: 0.7232 - val_loss: 0.7579 - val_acc: 0.7386
Epoch 8/

[0.4170594669133425, 0.945]

In [23]:
from pandas import DataFrame, concat
a = DataFrame(classifier.predict(X_test))
b = DataFrame(y_test)
a['paths_'] = [i.split('/')[5] for i in test]
a['paths'] = [i.split('/')[4] for i in test]
a= concat([a, b], axis=1)
print(a)


                0             1             2               paths_  \
0    9.994501e-01  9.889761e-01  1.908592e-04      cherry_0835.jpg   
1    9.969657e-01  9.950427e-01  3.077818e-03  strawberry_0496.jpg   
2    3.490219e-04  3.561716e-04  9.999553e-01      tomato_0115.jpg   
3    2.134736e-02  1.566077e-02  9.927487e-01      tomato_1453.jpg   
4    9.999989e-01  9.999863e-01  6.242022e-07  strawberry_1990.jpg   
5    9.244129e-01  1.850771e-01  2.618350e-02      cherry_1517.jpg   
6    1.000000e+00  1.000000e+00  4.454305e-20      cherry_1877.jpg   
7    1.000000e+00  1.000000e+00  1.399146e-13  strawberry_2016.jpg   
8    9.996374e-01  4.378458e-02  8.918051e-06      cherry_0793.jpg   
9    2.635427e-05  3.332852e-05  9.999981e-01      tomato_0654.jpg   
10   9.820614e-01  1.508209e-01  2.858717e-03      cherry_1463.jpg   
11   1.000000e+00  1.000000e+00  3.297778e-15      cherry_1613.jpg   
12   8.969576e-01  6.894299e-01  6.194155e-02      cherry_2267.jpg   
13   4.607813e-04  4

In [0]:
a.to_csv("./drive/My Drive/prj/predictions_third_dropout.csv")

Save Model

In [0]:
classifier.save("./drive/My Drive/prj/models/last.h5", 'w+')

Image noise generator:

In [0]:
transformer= ImageDataGenerator(
    zoom_range=0.2,  # randomly zoom into images
    rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=True  # randomly flip images
)

Train with generator

In [0]:
classifier.fit_generator(
    transformer.flow(X_train, y_train),
    epochs=10,
    verbose=1 # , callbacks=K.callbacks.CSVLogger('./drive/My Drive/prj/logs/', separator=',', append=False)
)
score_permu = classifier.evaluate(X_test, y_test, verbose=0)
score_permu

In [0]:
# classifier.save("./drive/My Drive/prj/models/first.h5", 'w+')